In [3]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 35 kB/s 
     |████████████████████████████████| 198 kB 48.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=c4d69a949eef711ca3395a98963410d88be81da932545300db30adbbfcaae170
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [56]:
# Importing Libraries
import numpy as np
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import col
from pyspark.sql.functions import *
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

In [38]:
appName= "hive_pyspark"
master= "local"
spark = SparkSession.builder.master(master).appName(appName).enableHiveSupport().getOrCreate()

### Question 1

In [40]:
# Reading the given CSV file in a Hive table
df=spark.read.csv("boston.csv",header=True)
df.show(10)

+-----------+----+-----------+----+-----------+-----------+-----------+-----------+---+---+-----------+-----------+-----------+-----------+
|       CRIM|  ZN|      INDUS|CHAS|        NOX|         RM|        AGE|        DIS|RAD|TAX|         PT|          B|      LSTAT|         MV|
+-----------+----+-----------+----+-----------+-----------+-----------+-----------+---+---+-----------+-----------+-----------+-----------+
|    0.00632|  18|2.309999943|   0|0.537999988|6.574999809|65.19999695|4.090000153|  1|296|15.30000019|396.8999939|4.980000019|         24|
|0.027310001|   0|7.070000172|   0|0.469000012|6.421000004|78.90000153|4.967100143|  2|242|17.79999924|396.8999939|9.140000343|21.60000038|
|    0.02729|   0|7.070000172|   0|0.469000012|7.184999943|61.09999847|4.967100143|  2|242|17.79999924|392.8299866| 4.03000021|34.70000076|
|0.032370001|   0|2.180000067|   0|0.458000004|6.998000145|45.79999924|6.062200069|  3|222|18.70000076|394.6300049|2.940000057|33.40000153|
|0.069049999|   0|2.

In [43]:
# Saving the table
df.write.saveAsTable("boston_table")

In [41]:
spark.sql("select * from boston_table limit 10").show()

+-----------+----+-----------+----+-----------+-----------+-----------+-----------+---+---+-----------+-----------+-----------+-----------+
|       CRIM|  ZN|      INDUS|CHAS|        NOX|         RM|        AGE|        DIS|RAD|TAX|         PT|          B|      LSTAT|         MV|
+-----------+----+-----------+----+-----------+-----------+-----------+-----------+---+---+-----------+-----------+-----------+-----------+
|    0.00632|  18|2.309999943|   0|0.537999988|6.574999809|65.19999695|4.090000153|  1|296|15.30000019|396.8999939|4.980000019|         24|
|0.027310001|   0|7.070000172|   0|0.469000012|6.421000004|78.90000153|4.967100143|  2|242|17.79999924|396.8999939|9.140000343|21.60000038|
|    0.02729|   0|7.070000172|   0|0.469000012|7.184999943|61.09999847|4.967100143|  2|242|17.79999924|392.8299866| 4.03000021|34.70000076|
|0.032370001|   0|2.180000067|   0|0.458000004|6.998000145|45.79999924|6.062200069|  3|222|18.70000076|394.6300049|2.940000057|33.40000153|
|0.069049999|   0|2.

### Question 2

In [44]:
# Reading the data from Hive table as spark dataframe
df1=spark.sql("select * from boston_table")
df1.show()

+-----------+----+-----------+----+-----------+-----------+-----------+-----------+---+---+-----------+-----------+-----------+-----------+
|       CRIM|  ZN|      INDUS|CHAS|        NOX|         RM|        AGE|        DIS|RAD|TAX|         PT|          B|      LSTAT|         MV|
+-----------+----+-----------+----+-----------+-----------+-----------+-----------+---+---+-----------+-----------+-----------+-----------+
|    0.00632|  18|2.309999943|   0|0.537999988|6.574999809|65.19999695|4.090000153|  1|296|15.30000019|396.8999939|4.980000019|         24|
|0.027310001|   0|7.070000172|   0|0.469000012|6.421000004|78.90000153|4.967100143|  2|242|17.79999924|396.8999939|9.140000343|21.60000038|
|    0.02729|   0|7.070000172|   0|0.469000012|7.184999943|61.09999847|4.967100143|  2|242|17.79999924|392.8299866| 4.03000021|34.70000076|
|0.032370001|   0|2.180000067|   0|0.458000004|6.998000145|45.79999924|6.062200069|  3|222|18.70000076|394.6300049|2.940000057|33.40000153|
|0.069049999|   0|2.

In [45]:
df1.printSchema()

root
 |-- CRIM: string (nullable = true)
 |-- ZN: string (nullable = true)
 |-- INDUS: string (nullable = true)
 |-- CHAS: string (nullable = true)
 |-- NOX: string (nullable = true)
 |-- RM: string (nullable = true)
 |-- AGE: string (nullable = true)
 |-- DIS: string (nullable = true)
 |-- RAD: string (nullable = true)
 |-- TAX: string (nullable = true)
 |-- PT: string (nullable = true)
 |-- B: string (nullable = true)
 |-- LSTAT: string (nullable = true)
 |-- MV: string (nullable = true)



In [65]:
from pyspark.sql.functions import col
df1 = df1.select([col(column).cast('double') for column in df1.columns])

In [46]:
df1.describe().show()

+-------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+-----------------+------------------+------------------+-----------------+
|summary|              CRIM|                ZN|             INDUS|              CHAS|               NOX|                RM|               AGE|               DIS|              RAD|               TAX|               PT|                 B|             LSTAT|               MV|
+-------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+-----------------+------------------+------------------+-----------------+
|  count|               506|               506|               506|               506|               506|               506|               506|               506|              506|  

In [47]:
df1.describe().toPandas().T

,0,1,2,3,4
summary,count,mean,stddev,min,max
CRIM,506,3.6135235608162057,8.601545086715594,0.00632,9.966540337
ZN,506,11.363636363636363,23.32245299451514,0,95
INDUS,506,11.136778749531626,6.86035298095724,0.460000008,9.899999619
CHAS,506,0.0691699604743083,0.2539940413404101,0,1
NOX,506,0.5546950602312246,0.1158776754570543,0.38499999,0.870999992
RM,506,6.28463438896641,0.7026171549511354,3.561000109,8.779999733
AGE,506,68.57490120115612,28.148861532793276,10,99.30000305
DIS,506,3.7950426960059325,2.105710142043288,1.129600048,9.222900391
RAD,506,9.549407114624506,8.707259384239366,1,8


### Question 3

In [66]:
df1.toPandas().corr()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PT,B,LSTAT,MV
CRIM,1.000000,-0.200469,0.406583,-0.055892,0.420972,-0.219247,0.352734,-0.379670,0.625505,0.582764,0.289946,-0.385064,0.455621,-0.388305
ZN,-0.200469,1.000000,-0.533828,-0.042697,-0.516604,0.311991,-0.569537,0.664408,-0.311948,-0.314563,-0.391679,0.175520,-0.412995,0.360445
INDUS,0.406583,-0.533828,1.000000,0.062938,0.763651,-0.391676,0.644779,-0.708027,0.595129,0.720760,0.383248,-0.356977,0.603800,-0.483725
CHAS,-0.055892,-0.042697,0.062938,1.000000,0.091203,0.091251,0.086518,-0.099176,-0.007368,-0.035587,-0.121515,0.048788,-0.053929,0.175260
NOX,0.420972,-0.516604,0.763651,0.091203,1.000000,-0.302188,0.731470,-0.769230,0.611441,0.668023,0.188933,-0.380051,0.590879,-0.427321
RM,-0.219247,0.311991,-0.391676,0.091251,-0.302188,1.000000,-0.240265,0.205246,-0.209847,-0.292048,-0.355502,0.128069,-0.613808,0.695360
AGE,0.352734,-0.569537,0.644779,0.086518,0.731470,-0.240265,1.000000,-0.747881,0.456022,0.506456,0.261515,-0.273534,0.602339,-0.376955
DIS,-0.379670,0.664408,-0.708027,-0.099176,-0.769230,0.205246,-0.747881,1.000000,-0.494588,-0.534432,-0.232471,0.291512,-0.496996,0.249929
RAD,0.625505,-0.311948,0.595129,-0.007368,0.611441,-0.209847,0.456022,-0.494588,1.000000,0.910228,0.464741,-0.444413,0.488676,-0.381626
TAX,0.582764,-0.314563,0.720760,-0.035587,0.668023,-0.292048,0.506456,-0.534432,0.910228,1.000000,0.460853,-0.441808,0.543993,-0.468536


In [67]:
mat=np.array(matrix.collect()[0]["pearson({})".format(vector_col)].values)

In [72]:
c=0
j=0
print("       ","   ".join(df1.columns))
for i in mat:
  if c==0:
    print("\n",df1.columns[j],end="    ")
    j+=1
  if c!=13:
    print("%.2f"%i,end ="  ")
    c+=1
  else:
    c=0

        CRIM   ZN   INDUS   CHAS   NOX   RM   AGE   DIS   RAD   TAX   PT   B   LSTAT   MV

 CRIM    1.00  -0.20  0.41  -0.06  0.42  -0.22  0.35  -0.38  0.63  0.58  0.29  -0.39  0.46  
 ZN    -0.20  1.00  -0.53  -0.04  -0.52  0.31  -0.57  0.66  -0.31  -0.31  -0.39  0.18  -0.41  
 INDUS    0.41  -0.53  1.00  0.06  0.76  -0.39  0.64  -0.71  0.60  0.72  0.38  -0.36  0.60  
 CHAS    -0.06  -0.04  0.06  1.00  0.09  0.09  0.09  -0.10  -0.01  -0.04  -0.12  0.05  -0.05  
 NOX    0.42  -0.52  0.76  0.09  1.00  -0.30  0.73  -0.77  0.61  0.67  0.19  -0.38  0.59  
 RM    -0.22  0.31  -0.39  0.09  -0.30  1.00  -0.24  0.21  -0.21  -0.29  -0.36  0.13  -0.61  
 AGE    0.35  -0.57  0.64  0.09  0.73  -0.24  1.00  -0.75  0.46  0.51  0.26  -0.27  0.60  
 DIS    -0.38  0.66  -0.71  -0.10  -0.77  0.21  -0.75  1.00  -0.49  -0.53  -0.23  0.29  -0.50  
 RAD    0.63  -0.31  0.60  -0.01  0.61  -0.21  0.46  -0.49  1.00  0.91  0.46  -0.44  0.49  
 TAX    0.58  -0.31  0.72  -0.04  0.67  -0.29  0.51  -0.53  0.91  1.0

In [73]:
import six
for i in df.columns:
    if not( isinstance(df1.select(i).take(1)[0][0], six.string_types)):
        print( "Correlation to MV for ", i, df1.stat.corr('MV',i))

Correlation to MV for  CRIM -0.3883046116575088
Correlation to MV for  ZN 0.36044534463752903
Correlation to MV for  INDUS -0.48372517128143383
Correlation to MV for  CHAS 0.17526017775291847
Correlation to MV for  NOX -0.4273207763683772
Correlation to MV for  RM 0.695359937127267
Correlation to MV for  AGE -0.37695456714288667
Correlation to MV for  DIS 0.24992873873512172
Correlation to MV for  RAD -0.3816262315669168
Correlation to MV for  TAX -0.46853593528654536
Correlation to MV for  PT -0.5077867038116085
Correlation to MV for  B 0.3334608226834164
Correlation to MV for  LSTAT -0.7376627294671615
Correlation to MV for  MV 1.0


### Question 4

In [74]:
from pyspark.sql.functions import isnull, when, count, col
df1.select([count(when(isnull(c), c)).alias(c) for c in df1.columns]).show()

+----+---+-----+----+---+---+---+---+---+---+---+---+-----+---+
|CRIM| ZN|INDUS|CHAS|NOX| RM|AGE|DIS|RAD|TAX| PT|  B|LSTAT| MV|
+----+---+-----+----+---+---+---+---+---+---+---+---+-----+---+
|   0|  0|    0|   0|  0|  0|  0|  0|  0|  0|  0|  0|    0|  0|
+----+---+-----+----+---+---+---+---+---+---+---+---+-----+---+



In [78]:
assemb = VectorAssembler(inputCols = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PT', 'B', 'LSTAT'], outputCol = 'features')

In [79]:
df_assemb = assemb.transform(df1)

In [80]:
df_assemb = df_assemb.select(['features', 'MV'])
df_assemb.show(3)

+--------------------+-----------+
|            features|         MV|
+--------------------+-----------+
|[0.00632,18.0,2.3...|       24.0|
|[0.027310001,0.0,...|21.60000038|
|[0.02729,0.0,7.07...|34.70000076|
+--------------------+-----------+
only showing top 3 rows



In [81]:
# Spliting the dataset 
train_df,test_df = df_assemb.randomSplit([0.7, 0.3])

In [82]:
# Building and fitting Linear Regression Model
lr = LinearRegression(featuresCol = 'features', labelCol='MV')
lr_model = lr.fit(train_df)

### Question 5

In [95]:
# Printing the RMSE & R2 Values of train data
trainingSummary = lr_model.summary
print("Root Mean Squared Error (RMSE) on train data = %f" % trainingSummary.rootMeanSquaredError)
print("R Squared (R2) on train data = %f" % trainingSummary.r2)

Root Mean Squared Error (RMSE) on train data = 4.499097
R Squared (R2) on train data = 0.738296


In [92]:
lr_pred = lr_model.transform(test_df)
lr_evaluator = RegressionEvaluator(predictionCol="prediction",labelCol="MV",metricName="r2")

In [91]:
# Comparing the Predicted and Visually
predictions = lr_model.transform(test_df)
predictions.select("prediction","MV").show()

+------------------+-----------+
|        prediction|         MV|
+------------------+-----------+
| 25.49510998674593|30.10000038|
| 42.61922067703623|       50.0|
| 25.01775300030161|24.70000076|
|36.480798553455784|42.29999924|
| 31.84717002257384|31.10000038|
|27.675046299882396|23.89999962|
| 30.36752958125172|34.70000076|
| 31.27828933415858|30.79999924|
|22.001402278463324|26.60000038|
|19.572792959132574|       18.5|
|16.794751295849995|       17.5|
|29.481982850308526|24.10000038|
| 23.34842759682373|19.39999962|
| 41.54136271711516|       48.5|
| 31.76452429864883|27.89999962|
| 34.45226686865157|34.59999847|
|27.200533432952525|23.20000076|
|28.794402286512522|22.89999962|
|25.212937848570675|23.29999924|
|27.283558295621443|22.60000038|
+------------------+-----------+
only showing top 20 rows



In [96]:
# Print R2 and RMSE for Test Data
test_result = lr_model.evaluate(test_df)
print("Root Mean Squared Error (RMSE) on test data = %g" % test_result.rootMeanSquaredError)
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(lr_pred))

Root Mean Squared Error (RMSE) on test data = 5.14257
R Squared (R2) on test data = 0.73404
